In [ ]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
import pandas as pd
from core.constants import TableKey, TableName

load_dotenv()
user_name = os.environ.get("sql_user")
password = os.environ.get("sql_password")
db_name = os.environ.get("sql_db_name")
sql_ip = os.environ.get("sql_ip")
sql_port = os.environ.get("sql_port")

db_url = f"mariadb+mariadbconnector://{user_name}:{password}@{sql_ip}:{sql_port}/"f"{db_name}"

engine = create_engine(db_url, pool_pre_ping=True, pool_recycle=3600)

conn = engine.connect()
meta = MetaData()
meta.reflect(conn)
meta.tables

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)

for table_name in inspector.get_table_names():
   for column in inspector.get_columns(table_name):
       print("Column: %s" % column['name'])

In [ ]:
from core.constants import TableKey, TableName

df = pd.read_csv('../data/company-profile-20220827.csv')

from sqlalchemy import types

column_types = {TableKey.Profile.SYMBOL: types.NVARCHAR(20),
                TableKey.Profile.SECTOR: types.NVARCHAR(40),
                TableKey.Profile.INDUSTRY: types.TEXT,
                TableKey.Profile.NAME: types.TEXT}

columns = (Column(k, v) for k, v in column_types.items())
table = Table(TableName.PROFILE, meta, *columns, extend_existing=True)
meta.create_all(engine, )

try:
    sql = f'alter table {TableName.PROFILE} add primary key({TableKey.Profile.SYMBOL})'
    conn.execute(sql)
except:
    print("Aleardy PK defined.")

from sqlalchemy.dialects.mysql import insert

stmt = insert(table).values([df.iloc[i][list(column_types.keys())].to_dict() for i in range(len(df))])
update_dict = {x.name: x for x in stmt.inserted}

on_duplicate_key_stmt = stmt.on_duplicate_key_update(update_dict)
conn.execute(statement=on_duplicate_key_stmt)


In [ ]:
# https://lowelllll.github.io/til/2019/04/19/TIL-flask-sqlalchemy-orm/
# result = db.session.execute('SELECT * FROM my_table WHERE my_column = :val', {'val': 5})
sql = f"select {TableKey.Profile.SYMBOL} from {TableName.PROFILE} WHERE {TableKey.Profile.SECTOR} = 'Industrials';"
result = conn.execute(sql)

In [ ]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()

apikey = os.environ.get("apikey")

In [24]:
# https://lowelllll.github.io/til/2019/04/19/TIL-flask-sqlalchemy-orm/
# result = db.session.execute('SELECT * FROM my_table WHERE my_column = :val', {'val': 5})
sql = f"select {TableKey.Profile.SYMBOL} from {TableName.PROFILE} WHERE {TableKey.Profile.SECTOR} = 'Industrials';"
result = conn.execute(sql)

In [ ]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
from tqdm import tqdm
import os

load_dotenv()

apikey = os.environ.get("apikey")